In [1]:
import sys
sys.path.append('../tools')

import os

from extract_features_from_lyric import load_full_lyric_dataset

Using TensorFlow backend.


In [2]:
dataset_path = os.path.join('..', '..', 'database', 'lyrics')
database_path = os.path.join('..', 'database', 'MoodyLyrics4Q_cleaned_split.csv')

dataset = load_full_lyric_dataset(dataset_path, database_path)

Detected lang = so
Detected lang = it
Song ML1056 is duplicated. 
For ..\..\database\lyrics\ML1068.json there is no language info in dataset
Detected lang = en
For ..\..\database\lyrics\ML1098.json there is no language info in dataset
Detected lang = en
Detected lang = en
For ..\..\database\lyrics\ML1159.json lyric is empty
For ..\..\database\lyrics\ML1159.json there is no language info in dataset
No features in text.
Detected lang = None
For ..\..\database\lyrics\ML1159.json is instrumental

For ..\..\database\lyrics\ML1230.json lyric is empty
For ..\..\database\lyrics\ML1230.json there is no language info in dataset
No features in text.
Detected lang = None
For ..\..\database\lyrics\ML1230.json is instrumental

For ..\..\database\lyrics\ML1291.json there is no language info in dataset
Detected lang = en
For ..\..\database\lyrics\ML1333.json there is no language info in dataset
Detected lang = de
For ..\..\database\lyrics\ML1336.json lyric is empty
For ..\..\database\lyrics\ML1336.jso

In [3]:
dataset.head()

,mood,title,lyric,language,instrumental,split
ML1,happy,I Want Your Sex,I Want Your Sex Lyrics[From a PSA recorded for...,en,False,train
ML10,happy,Heart of Glass,Heart of Glass Lyrics[Verse 1]\nOnce I had a l...,en,False,train
ML100,happy,Crazy Little Thing Called Love,Crazy Little Thing Called Love Lyrics[Intro]\n...,en,False,train
ML1000,happy,Almost,Almost Lyrics[Verse 1]\nI almost got drunk at ...,en,False,train
ML1001,happy,Glow,Glow Lyrics[Verse 1]\nI never thought that you...,en,False,train


In [4]:
happy = dataset[dataset['mood'] == 'happy']
sad = dataset[dataset['mood'] == 'sad']
angry = dataset[dataset['mood'] == 'angry']
relaxed = dataset[dataset['mood'] == 'relaxed']


In [5]:
happy.value_counts('split')

split
train    350
test      75
val       75
dtype: int64

In [6]:
en_dataset = dataset[dataset['language'] == 'en']

en_happy = en_dataset[en_dataset['mood'] == 'happy']
en_sad = en_dataset[en_dataset['mood'] == 'sad']
en_angry = en_dataset[en_dataset['mood'] == 'angry']
en_relaxed = en_dataset[en_dataset['mood'] == 'relaxed']

In [7]:
en_happy.value_counts('split')

split
train    331
test      72
val       68
dtype: int64

In [8]:
en_happy['lyric'].head()

ML1       I Want Your Sex Lyrics[From a PSA recorded for...
ML10      Heart of Glass Lyrics[Verse 1]\nOnce I had a l...
ML100     Crazy Little Thing Called Love Lyrics[Intro]\n...
ML1000    Almost Lyrics[Verse 1]\nI almost got drunk at ...
ML1001    Glow Lyrics[Verse 1]\nI never thought that you...
Name: lyric, dtype: object

In [9]:
from extract_features_from_lyric import clean_lyric

In [10]:
def cleaning_lyric(dataset):

    for index, row in dataset.iterrows():
        lyric, _ = clean_lyric(row['lyric'], row['title'])
        dataset.at[index, 'lyric'] = lyric

    return dataset


en_happy = cleaning_lyric(en_happy)
en_sad = cleaning_lyric(en_sad)
en_angry = cleaning_lyric(en_angry)
en_relaxed = cleaning_lyric(en_relaxed)


In [11]:
from transformers import XLNetTokenizer
import numpy as np


c:\Users\janto\anaconda3\envs\emotions\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
def get_statistics(num_tokens):
    print(f'Mean: {np.round(np.mean(num_tokens), 2)}')
    print(f'Median: {np.median(num_tokens)}')
    print(f'Max : {np.max(num_tokens)}')
    print(f'Min : {np.min(num_tokens)}')
    print(f'Std : {np.round(np.std(num_tokens), 2)}')

def count_tokens(lyrics, tokenizer):
    return [len(tokenizer(lyric)['input_ids']) for lyric in lyrics]

def get_statistics_for_category(dataset, tokenizer):
    num_tokens = count_tokens(dataset['lyric'], tokenizer)
    get_statistics(num_tokens)

def get_all_statistics_tokens(tokenizer):
    print('Happy')
    get_statistics_for_category(en_happy, tokenizer)

    print('Angry')
    get_statistics_for_category(en_angry, tokenizer)

    print('Sad')
    get_statistics_for_category(en_sad, tokenizer)

    print('Relaxed')
    get_statistics_for_category(en_relaxed, tokenizer)

In [17]:
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
get_all_statistics_tokens(tokenizer)

Happy
Mean: 375.96
Median: 340.0
Max : 1432
Min : 11
Std : 198.84
Angry
Mean: 342.63
Median: 273.0
Max : 1321
Min : 56
Std : 224.76
Sad
Mean: 228.97
Median: 206.0
Max : 1199
Min : 12
Std : 134.74
Relaxed
Mean: 230.21
Median: 201.0
Max : 872
Min : 10
Std : 135.04


In [14]:
def split_into_words(lyric):
    return lyric.split()

def get_statistics(num_words):
    print(f'Mean: {np.round(np.mean(num_words), 2)}')
    print(f'Max : {np.max(num_words)}')
    print(f'Min : {np.min(num_words)}')
    print(f'Std : {np.round(np.std(num_words), 2)}')

def count_words(lyrics):
    return [len(lyric.split()) for lyric in lyrics]

def get_statistics_for_category(dataset):
    num_words = count_words(dataset['lyric'])
    get_statistics(num_words)

def get_all_statistics_words():
    print('Happy')
    get_statistics_for_category(en_happy)

    print('Angry')
    get_statistics_for_category(en_angry)

    print('Sad')
    get_statistics_for_category(en_sad)

    print('Relaxed')
    get_statistics_for_category(en_relaxed)

In [15]:
get_all_statistics_words()

Happy
Mean: 316.58
Max : 1174
Min : 3
Std : 163.37
Angry
Mean: 288.37
Max : 1075
Min : 46
Std : 181.17
Sad
Mean: 203.95
Max : 1091
Min : 10
Std : 120.36
Relaxed
Mean: 202.45
Max : 733
Min : 6
Std : 116.97
